In [2]:
# warning 제거
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [3]:
import pandas as pd
import os

os.chdir(r"/Users/sanghyuk/Documents/preprocessing_python/lecture_source/5. 머신러닝 모델의 성능 향상을 위한 전처리/데이터/")

In [13]:
df = pd.read_csv("Telco_churn_prediction.csv")

In [14]:
# 대략적인 변수 확인
pd.set_option('display.max_columns', None) # 모든 컬럼 다보기
pd.set_option('display.max_rows', 10) # 행 열개까지만 보여주기
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [15]:
# ID는 삭제해야 함
# 특징과 라벨 분리
X = df.drop(['customerID', 'Churn'], axis = 1)
Y = df['Churn']

In [16]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [17]:
# 결측치 확인
Train_X.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
                   ..
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Length: 19, dtype: int64

In [18]:
# 특징별 타입 확인 => TotalCharges가 이상하다?
Train_X.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
                     ...   
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Length: 19, dtype: object

In [20]:
sorted(Train_X['TotalCharges'].unique())[:5] # 공백으로 표현된 결측이 포함되어있음을 확인

[' ', '100.25', '100.35', '100.4', '100.9']

In [21]:
# 결측이 공백으로 표현되어 있으니, 분석이 어려울 수 있음
# 모든 공백을 결측으로 변환
import numpy as np
for col in Train_X.columns:
    Train_X[col].replace(' ', np.nan, inplace = True)
    Test_X[col].replace(' ', np.nan, inplace = True)

In [22]:
# 결측 재확인
Train_X.isnull().sum().sort_values(ascending = False) # TotalCharges에만 결측이 있음을 확인

# 하나의 컬럼에만 결측이 있으며, 결측 개수가 많지 않음 
# 그러나 행을 삭제하기에는 무리가 있음 => 새로운 데이터에도 결측이 없으리란 보장이 없기 때문

TotalCharges        9
OnlineSecurity      0
SeniorCitizen       0
Partner             0
Dependents          0
                   ..
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
gender              0
Length: 19, dtype: int64

In [23]:
# 결측치 예측 모델을 사용하기로 결정
# 근거: TotalCharges는 사용 서비스, monthlycharges 등에 영향을 받을 수 밖에 없을 것이라 판단하기 때문
# 단, 범주형 변수를 처리할 때 Total Charges는 당연히 범주형으로 취급되면 안되므로, float으로 바꿔놓기

Train_X['TotalCharges'] = Train_X['TotalCharges'].astype(float)

In [24]:
# 범주형 변수 파악
category_columns = Train_X.select_dtypes(include = 'object').columns.tolist()
# OHE가 입력을 list로 받아서 tolist를 사용하여 자료형을 변경함

In [27]:
category_columns

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [30]:
# 범주형 변수 더미화
from feature_engine.encoding import OneHotEncoder as OHE
make_dummy_model = OHE(variables = category_columns, drop_last = True).fit(Train_X)

Train_X = make_dummy_model.transform(Train_X)
Test_X = make_dummy_model.transform(Test_X)

In [31]:
# 결측치 예측 모델 활용
# KNN Imputer 인스턴스화
from sklearn.impute import KNNImputer
KI = KNNImputer(n_neighbors = 5)

# KNN Imputer 학습
KI.fit(Train_X)

# 결측 대체
Train_X = pd.DataFrame(KI.transform(Train_X), columns = Train_X.columns)
Test_X = pd.DataFrame(KI.transform(Test_X), columns = Test_X.columns)

In [33]:
# 라벨 분포 확인
# 문자로 되어 있음을 확인 
# 매우 약한 클래스 불균형이 어느정도 있음
Train_Y.value_counts()

No     3918
Yes    1364
Name: Churn, dtype: int64

In [34]:
Train_Y.replace({"No":-1, "Yes":1}, inplace = True)
Test_Y.replace({"No":-1, "Yes":1}, inplace = True)

범주형과 이진형의 전처리가 다른 경우 있어서 보통 나눈다. 

In [35]:
# 유니크한 값의 개수를 바탕으로 연속형과 이진형 변수 구분
# 현재 one-hot까지 다 들어가있는 형태
continuous_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) > 3]
binary_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) <= 3]

In [37]:
# 이상치 확인
# 아웃라이어라는 것 자체가 당연히, 범주형데이터에 대해서는 의미가 없다. 
def IQR_rule(val_list): # 한 특징에 포함된 값 (열 벡터)
    # IQR 계산    
    Q1 = np.quantile(val_list, 0.25)
    Q3 = np.quantile(val_list, 0.75)
    IQR = Q3 - Q1
    
    # IQR rule을 위배하지 않는 bool list 계산 (True: 이상치 X, False: 이상치 O)
    not_outlier_condition = (Q3 + 1.5 * IQR > val_list) & (Q1 - 1.5 * IQR < val_list)
    return not_outlier_condition

num_outliers = len(Train_X) - Train_X[continuous_cols].apply(IQR_rule).sum(axis = 0) 
print(num_outliers) # 특별한 이상치 없음 => 제거 X

tenure            0
MonthlyCharges    0
TotalCharges      0
dtype: int64


이진형은 치우침도 의미가 없다. 설령 치우쳤어도, 해결할 방법이 없다. 너무 많이 치우쳐있으면, 그 feature자체를 빼고 그럴수밖에 없음. 


In [20]:
# 특징 치우침 확인 (주의: 이진형 변수는 치우쳤더라도 변환 자체가 의미가 없음. 따라서 연속형 변수만 보면 됨)
# 모든 연속형 변수 모두 치우침이 특별히 없어보임
Train_X[continuous_cols].skew() 

tenure            0.245591
MonthlyCharges   -0.190702
TotalCharges      0.984331
dtype: float64

이래서, 학습하기 직전에 shape을 확인하는 것. 그 전에는 shape을 알아도 뭐 어떻게 할꺼야. 

In [38]:
# 모델 선택을 위한 데이터 크기 확인
Train_X.shape 

# 샘플대비 특징이 많지 않음. 샘플이 충분히 있음. 특징이 대부분 이진형임 (더미화를 했으므로) ==> 복잡한 모델 사용이 적절
# 앙상블 계열의 모델을 사용하기로 결정: 근거 => 이진형 특징들이 더 많고, 각 특징이 제대로 반영되면 좋은 성능을 낼 수 있을 것이라 기대해서

(5282, 30)

In [39]:
# 특징 선택과 모델 하이퍼 파라미터 튜닝
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGB
from lightgbm import LGBMClassifier as LGBM
from sklearn.feature_selection import *
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score

# 모델 파라미터 그리드 설계
model_parameter_grid = dict()
model_parameter_grid[RFC] = ParameterGrid({"max_depth": [2, 3, 4],
                                          "n_estimators": [50, 100]})

model_parameter_grid[XGB] = ParameterGrid({"max_depth": [2, 3, 4],
                                          "n_estimators": [50, 100],
                                          "learning_rate": [0.05, 0.1, 0.15, 0.2]})

model_parameter_grid[LGBM] = ParameterGrid({"max_depth": [2, 3, 4],
                                          "n_estimators": [50, 100],
                                           "learning_rate": [0.05, 0.1, 0.15, 0.2]})

In [40]:
# f_regression(X, Y) => (statistics, p-value)
continous_cols_pvals = f_classif(Train_X[continuous_cols], Train_Y)[1]
binary_cols_pvals = chi2(Train_X[binary_cols], Train_Y)[1]

In [42]:
continous_cols_pvals, binary_cols_pvals

(array([1.03871802e-146, 4.39781726e-049, 1.88796647e-042]),
 array([1.62963070e-28, 7.92024494e-01, 3.20479041e-15, 1.31879221e-12,
        8.94011529e-01, 2.44378304e-02, 6.86501893e-01, 7.83266036e-66,
        5.38310883e-14, 1.65039141e-25, 4.08967904e-71, 5.87878364e-46,
        2.64265325e-05, 9.66240140e-42, 3.86302494e-04, 1.16580337e-66,
        1.14231349e-21, 2.73766343e-11, 1.08890108e-05, 1.14637187e-13,
        2.74288996e-04, 2.27858893e-82, 3.48383682e-78, 5.98089862e-22,
        2.62503147e-76, 4.19703109e-18, 2.45257783e-10]))

In [43]:
# 각각을 Series로 변환 (value: pvalue, index: colum name)
cont_pvals = pd.Series(continous_cols_pvals, index = continuous_cols)
binary_pvals = pd.Series(binary_cols_pvals, index = binary_cols)

In [44]:
# cont_pvals과 binary_pvals을 합친 뒤, 오름차순으로 정렬 (앞에 나오는 특징부터 좋은 특징)
pvals = pd.concat([cont_pvals, binary_pvals])
pvals.sort_values(ascending = True, inplace = True)

In [45]:
#pvals 가장 높은 것부터 정렬되어 있는 상태. 

pvals

tenure                            1.038718e-146
Contract_Month-to-month            2.278589e-82
Contract_Two year                  3.483837e-78
PaymentMethod_Electronic check     2.625031e-76
OnlineSecurity_No                  4.089679e-71
                                      ...      
DeviceProtection_Yes               3.863025e-04
MultipleLines_No                   2.443783e-02
MultipleLines_No phone service     6.865019e-01
gender_Female                      7.920245e-01
PhoneService_Yes                   8.940115e-01
Length: 30, dtype: float64

In [46]:
# 튜닝 시작
best_score = 0
for k in range(30, 5, -1): 
    print(k)
    s_Train_X = Train_X[pvals.iloc[:k].index]
    s_Test_X = Test_X[pvals.iloc[:k].index]
    for M in model_parameter_grid.keys():
        for P in model_parameter_grid[M]:
            model = M(**P).fit(s_Train_X, Train_Y)
            pred_Y = model.predict(s_Test_X)
            score = f1_score(Test_Y, pred_Y)
            if score > best_score:
                best_score = score
                best_feature = s_Train_X.columns
                best_model = M
                best_parameter = P    

30
[01:43:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[01:43:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[01:43:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[01:43:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:43:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

25
[01:44:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:44:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:44:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:44:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

...

[01:45:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

10
[01:45:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[01:45:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[01:45:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[01:45:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:45:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

In [47]:
best_model, best_parameter, best_feature

(lightgbm.sklearn.LGBMClassifier,
 {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50},
 Index(['tenure', 'Contract_Month-to-month', 'Contract_Two year',
        'PaymentMethod_Electronic check', 'OnlineSecurity_No', 'TechSupport_No',
        'InternetService_Fiber optic', 'MonthlyCharges', 'OnlineBackup_No',
        'TotalCharges', 'DeviceProtection_No', 'SeniorCitizen',
        'OnlineSecurity_Yes', 'PaperlessBilling_Yes', 'TechSupport_Yes',
        'PaymentMethod_Credit card (automatic)', 'Partner_Yes',
        'InternetService_DSL', 'StreamingMovies_No', 'Dependents_No',
        'StreamingTV_No', 'PaymentMethod_Mailed check', 'StreamingTV_Yes',
        'OnlineBackup_Yes', 'StreamingMovies_Yes', 'DeviceProtection_Yes'],
       dtype='object'))